In [1]:
import numpy as np
from numpy import exp, sqrt, abs
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
plt.rcParams['font.size'] = 13

In [2]:
dname="DATA"; fname=dname+'/x_RBM_2024_lesson.dat'
v = np.loadtxt(fname, delimiter=" ",dtype=int)
v0 = np.copy(v)
N = len(v)
L = len(v[1])
SPINS = False

if SPINS:
    GAP=2
    v = 2*v - 1
    vmin=-1
else:
    GAP=1
    vmin=0

In [4]:
Ms = range(1, 7)
l_rate = 1.0

In [5]:
def activate(v_in, wei, bias, DE):
    act = np.dot(v_in, wei) + bias
    prob = 1. / (1. + exp(-DE * act))
    v_out = np.full(act.shape, vmin, dtype=int)
    v_out[np.random.rand(*act.shape) < prob] = 1
    if info:
        print('input=', v_in)
        print('act=',act)
        print('prob=',prob)
        print('output=',v_out)
    return v_out

In [8]:
Ms = range(1, 7)  # M values from 1 to 6
models = []

for M in Ms:
    print('===================================================')
    print(f"Training with M = {M} hidden units")

    sigma = sqrt(4. / (L + M))
    np.random.seed(12345)
    w = sigma * np.random.randn(L, M)
    a = sigma * np.random.randn(L)
    b = np.zeros(M)

    l_rate = 1.0
    mini = 500
    mini_batch_counter = 0

    for epoch in range(100):
        l_rate_m = l_rate / mini
        for k in range(N):
            if mini_batch_counter == 0:
                v_data, v_model = np.zeros(L), np.zeros(L)
                h_data, h_model = np.zeros(M), np.zeros(M)
                vh_data, vh_model = np.zeros((L, M)), np.zeros((L, M))

            h = activate(v0[k], w, b, GAP)
            vf = activate(h, w.T, a, GAP)
            hf = activate(vf, w, b, GAP)

            v_data += v0[k]
            v_model += vf
            h_data += h
            h_model += hf
            vh_data += np.outer(v0[k], h)
            vh_model += np.outer(vf, hf)

            mini_batch_counter += 1
            if mini_batch_counter == mini:
                dw = l_rate_m * (vh_data - vh_model)
                da = l_rate_m * (v_data - v_model)
                db = l_rate_m * (h_data - h_model)

                w += dw
                a += da
                b += db

                mini_batch_counter = 0

        np.random.shuffle(v0)
        l_rate = l_rate / (0.01 * l_rate + 1)
        if epoch % 10 == 9:
            print(f'Epoch {epoch + 1}, Learning Rate = {l_rate:.4f}')
    # Store trained model parameters
    models.append((w, a, b))

Training with M = 1 hidden units
Epoch 10, Learning Rate = 0.9091
Epoch 20, Learning Rate = 0.8333
Epoch 30, Learning Rate = 0.7692
Epoch 40, Learning Rate = 0.7143
Epoch 50, Learning Rate = 0.6667
Epoch 60, Learning Rate = 0.6250
Epoch 70, Learning Rate = 0.5882
Epoch 80, Learning Rate = 0.5556
Epoch 90, Learning Rate = 0.5263
Epoch 100, Learning Rate = 0.5000
Finished training for M=1 with final learning rate = 0.50000
Training with M = 2 hidden units
Epoch 10, Learning Rate = 0.9091
Epoch 20, Learning Rate = 0.8333
Epoch 30, Learning Rate = 0.7692
Epoch 40, Learning Rate = 0.7143
Epoch 50, Learning Rate = 0.6667
Epoch 60, Learning Rate = 0.6250
Epoch 70, Learning Rate = 0.5882
Epoch 80, Learning Rate = 0.5556
Epoch 90, Learning Rate = 0.5263
Epoch 100, Learning Rate = 0.5000
Finished training for M=2 with final learning rate = 0.50000
Training with M = 3 hidden units
Epoch 10, Learning Rate = 0.9091
Epoch 20, Learning Rate = 0.8333
Epoch 30, Learning Rate = 0.7692
Epoch 40, Learning